In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
batch_size = 128
num_classes = 10
epochs = 1

In [3]:
img_rows, img_cols = 28, 28

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

下面的代码会做一个判断，因为对于图像的编码有两种形式，一种是通道前置，另一种是通道后置，所以需要处理

In [5]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

下面的代码对数据集做了简单的处理，首先规范化数值类型为float32位，之后进行归一化操作，因为一个点是$2^{8}$的值，归一化能更好的训练

In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


下面的代码是利用keras自带的函数进行one hot编码

In [7]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [8]:
y_train.shape

(60000, 10)

In [9]:
y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

**线性模型**

In [10]:
seqmodel = Sequential()

In [11]:
seqmodel.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
seqmodel.add(Conv2D(64, (3, 3), activation='relu'))
seqmodel.add(MaxPooling2D(pool_size=(2, 2)))
seqmodel.add(Dropout(0.25))
seqmodel.add(Flatten())
seqmodel.add(Dense(128, activation='relu'))
seqmodel.add(Dropout(0.5))
seqmodel.add(Dense(num_classes, activation='softmax'))

In [12]:
seqmodel.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [13]:
seqmodel.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = seqmodel.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 65s 1ms/step - loss: 0.2706 - acc: 0.9164 - val_loss: 0.0687 - val_acc: 0.9777
Test loss: 0.06865063024163247
Test accuracy: 0.9777


**函数式模型**

In [14]:
from keras.layers import Input
from keras.models import Model

In [41]:
input_img = Input(shape=input_shape)

In [42]:
tower_1 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
tower_1 = Conv2D(64, (3, 3), padding='same', activation='relu')(tower_1)

tower_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
tower_2 = Conv2D(64, (5, 5), padding='same', activation='relu')(tower_2)

tower_3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input_img)
tower_3 = Conv2D(64, (1, 1), padding='same', activation='relu')(tower_3)

tower_4 = Conv2D(64,(1,1), padding='same',activation='relu')(input_img)

In [43]:
output = keras.layers.concatenate([tower_1, tower_2, tower_3,tower_4], axis=1)

In [44]:
tower_1_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(output)
tower_1_2 = Conv2D(64, (3, 3), padding='same', activation='relu')(tower_1_2)

tower_2_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(output)
tower_2_2 = Conv2D(64, (5, 5), padding='same', activation='relu')(tower_2_2)

tower_3_2 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(output)
tower_3_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(tower_3_2)

tower_4_2 = Conv2D(64,(1,1), padding='same',activation='relu')(output)

In [45]:
output_2 = keras.layers.concatenate([tower_1_2, tower_2_2, tower_3_2,tower_4_2], axis=1)

In [46]:
output_2 = Flatten()(output_2)
output_2 = Dense(64, activation='relu')(output_2)
output_2 = Dropout(0.5)(output_2)
output_2 = Dense(num_classes, activation='softmax')(output_2)

In [47]:
from keras.callbacks import EarlyStopping

In [48]:
funmodel = Model(inputs=input_img, outputs=output_2)

In [49]:
funmodel.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [50]:
tb = TensorBoard(log_dir="./log")

In [40]:
funmodel.fit(x_train, y_train,
            batch_size=batch_size,
            epochs= 1,
            verbose=1,
            validation_data=(x_test, y_test),
            callbacks=[EarlyStopping(patience=3,mode='auto',verbose=1),
                        TensorBoard(log_dir="./log")])
score = funmodel.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
  768/60000 [..............................] - ETA: 2:33:04 - loss: 3.8805 - acc: 0.1276

KeyboardInterrupt: 

In [25]:
from keras.callbacks import TensorBoard